In [ ]:
# prepare packages
import requests, pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import logging
# from pythonjsonlogger import jsonlogger
import logmatic
import os
from pymongo import MongoClient

import time, random
# 現狀 抓2~10頁 並塞入DB

In [ ]:
# mongoDB Connector
from Mongo_account import MongoBase

mongo = MongoBase("test3")

In [ ]:
mongo.collection.find({})

In [ ]:

# 設定現在時間 以時間來作為log檔名 一個log 以日計算
# 並在當下目錄創建 log folder
def get_time():
    now = datetime.utcnow()
    path = "./log"

    utc_8 = timezone(timedelta(hours=8))
    now = now.replace(tzinfo=timezone.utc)

    time_now = now.astimezone(utc_8)

    split = str(time_now).split(" ")
    file_date = split[0]
    split = split[1].split(":")
    file_time = split[0]+split[1]
    if not os.path.exists(path) :
        os.makedirs(path)
        
    return path + "/" + file_date 




In [ ]:
# memory
#url = "https://www.ptt.cc/bbs/Food/index2.html"

# set proxy to use
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [ ]:
# example use of logmatic 
# more simple 

logger = logging.getLogger()
 
handler = logging.FileHandler(get_time())
handler.setFormatter(logmatic.JsonFormatter())
 
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [ ]:
# 製作 url List
def get_url(n=2,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.ptt.cc/bbs/Food/index" + str(t) + ".html"
        result.append(url)
    return result

In [ ]:
#new_result


In [ ]:
# 連線流程
def get_connect(url):
    i = 0
    resp = ""
    try :
        resp = requests.get(url,headers=header,proxies=proxylist[i])
        resp.encoding = "utf-8"
        time.sleep(random.randint(1,4))
        # 設定連線狀況 log
        code = "HTTP response code = " + str(resp.status_code)
        logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds()})
        while resp.status_code != 200 :
            logger.info("Connect error",extra={"url" : url, "http response": resp.raise_for_status()})
            resp.close()
            i = (i + 1) % 6
            resp = requests.get(url,headers=header,proxies=proxylist[i])
            resp.encoding = "utf-8"



    except Exception as e:
        logger.error(e,exc_info=True)
    
    return resp.text 
    
    

In [ ]:
def parse(urls):
# 進入ptt列表 抓取 標題、推文數、網址 
    result = []
    soup = BeautifulSoup(get_connect(urls))
    print("進入ptt : ",urls)
    post = soup.find_all("div", class_="r-ent")
    for r in post:
        # 記憶區 保平安用
        # 多次調用條件中的值
        # 有時會出錯
        article = []
        href = ""
        title = ""
        author = ""
        title_in_content = ""
        date_in_content = ""
        pos = []
        neg = []
        arrow = []
        score = 0
        content = ""
        # 取得推的數量
        push = 0
        if r.find("div", class_="nrec").text:
            try:
                push = int(r.find("div", class_="nrec").text)
            except ValueError:
                    pass
        if r.find("a"):
            # 取得網址
            href = "http://www.ptt.cc" + r.find("a")["href"]
            # 取得標題
            title = r.find("a").text

            # 進入第二層 進入每一篇
            # 去除公告 請益 為分類的文章
            if not "公告" in title :
                if not "請益" in title:
                    print("現在處理: ", href)
                    art = BeautifulSoup(get_connect(href))
                    
                    content = art.find("div", id="main-content")
                    # 保存要丟棄的資訊
                    val = content.find_all("span", {"class":"article-meta-value"})
                    #  文章排版問題 有些文章沒有部份以下資訊
                    try:
                        author = val[0].text
                        title_in_content = val[2].text
                        date_in_content = val[3].text
                    except IndexError as e:
                        logger.warning("article span list out of index", extra = {"url" : href})
                        pass

                    # 開始丟棄資訊
                    removes = content.find_all("div", class_="article-metaline")
                    for remove in removes:
                        remove.extract()
                    removes = content.find_all("div", class_="article-metaline-right")
                    for remove in removes:
                        remove.extract()
                    removes = content.find_all("span", class_="f2")                    
                    for remove in removes:
                        if "※" in remove.text:
                            remove.extract()
                    # 處理推噓文
                    ps = content.find_all("div", class_="push")
                    for p in ps:
                        tag = p.find("span", class_="push-tag").text
                        if "推" in tag:
                            score = score + 1
                            # replace 第三個參數 只置換第一個
                            #push_content = p.find("span", class_="push-content").text.replace(": ","",1)
                            push_content = p.find("span", class_="push-content").text
                            push_ID = p.find("span", class_="push-userid").text
                            push_total = push_ID + push_content
                            pos.append(push_total)
                        elif "噓" in tag:
                            score = score - 1
                            push_content = p.find("span", class_="push-content").text
                            push_ID = p.find("span", class_="push-userid").text
                            push_total = push_ID + push_content
                            neg.append(push_total)
                        else :
                            push_content = p.find("span", class_="push-content").text
                            push_ID = p.find("span", class_="push-userid").text
                            push_total = push_ID + push_content
                            arrow.append(push_total)
                        p.extract()
                    #article.append({
                    #    "author" : author,
                    #    "title_in_content" : title_in_content,
                    #    "date_in_content" : date_in_content,                        
                    #    "content" : content.text,
                    #    "score" : score,
                    #    "推文" : pos,
                    #    "噓文" : neg                        
                    #})

        result.append({
            "title" : title,
            "author" : author,
            "date" : date_in_content,
            "push" : push,
            "href" : href,
            "content" : content.text,
            "score" : score,
            "推文" : pos,
            "噓文" : neg,
            "箭頭" : arrow
            

        })
        

    return result








In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait, ProcessPoolExecutor, as_completed

new_result = []
with ProcessPoolExecutor(max_workers= 5) as executor:
    future_result = {executor.submit(parse, url) : url for url in get_url(2,5)}

    

    for future in as_completed(future_result):
        new_result.append(future.result())
        print("work has done")
    
    

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
for r in new_result:
    for evey in r:
        s = pd.Series([evey["title"],evey["author"],evey["date"],evey["push"],evey["href"],evey["content"],evey["score"],evey["推文"],evey["噓文"],evey["箭頭"]]
                      ,index=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
        df = df.append(s,ignore_index=True)
        

In [ ]:
df



In [ ]:
import json

collect = json.loads(df.T.to_json()).values()

In [ ]:
from pymongo import MongoClient

class MongoBase:
    def __init__(self,collection):
        self.collection=collection
        self.OpenDB()
    def OpenDB(self):
        user='root'
        passwd='root'
        # on local host
        host='192.168.114.130'
        port='27017'
        auth_db='admin'
        uri = "mongodb://"+user+":"+passwd+"@"+host+":"+port+"/"+auth_db+"?authMechanism=SCRAM-SHA-1"
        self.con = MongoClient(uri, connect=True)
        self.db=self.con['wangdong']
        self.collection=self.db[self.collection]
    def closeDB(self):
        self.con.close()
        
mongo = MongoBase('test3')

In [ ]:


mongo.collection.insert_many(collect)
mongo.closeDB()


In [ ]:
# 進入ptt列表 抓取 標題、推文數、網址 
result = []
soup = BeautifulSoup(get_connect(url))
print("進入ptt : ",url)
post = soup.find_all("div", class_="r-ent")
for r in post:
            # 取得推的數量
    push = 0
    if r.find("div", class_="nrec").text:
        try:
            push = int(r.find("div", class_="nrec").text)
        except ValueError:
                pass
    if r.find("a"):
        # 取得網址
        href = "http://www.ptt.cc" + r.find("a")["href"]
        # 取得標題
        title = r.find("a").text
        
        # 進入第二層 進入每一篇
        # 去除公告 請益 為分類的文章
        if not "公告" in title :
            if not "請益" in title:
                print("現在處理: ", href)
                art = BeautifulSoup(get_connect(href))
                article = []
                content = art.find("div", id="main-content")
                # 保存要丟棄的資訊
                val = content.find_all("span", {"class":"article-meta-value"})
                author = val[0].text
                title_in_content = val[2].text
                date_in_content = val[3].text
                
                # 開始丟棄資訊
                removes = content.find_all("div", class_="article-metaline")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="article-metaline-right")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("span", class_="f2")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="push")
                for remove in removes:
                    remove.extract()
                article.append({
                    "author" : author,
                    "title_in_content" : title_in_content,
                    "date_in_content" : date_in_content,
                    "content" : content.text
                })
                
    result.append({
        "title" : title,
        "push" : push,
        "href" : href,
        "content" : article
        
    })
    
    time.sleep(random.randint(1,6))
                






In [ ]:
import json
# 轉成 Dataframe
df = pd.DataFrame(result)



In [ ]:
# 排列欄位順序

df = df[ ["title", "push", "href", "content" ] ]

In [ ]:
df.iloc[1]

In [ ]:
# I/O 較多 使用 thread
start = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(int, ['1', '2', '3', '4', '5'])
    for v in results:
        print(v)
end = time.time()
print("time used :",end - start)

In [ ]:
# example use of json_logging
# more detail
import logging
import sys

import json_logging

# log is initialized without a web framework name
json_logging.ENABLE_JSON_LOGGING = True
json_logging.init()

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

#logger.info("test log statement")
#logger.info("test log statement", extra={'props': {"extra_property": 'extra_value'}})

In [ ]:
# set logging 
# 設定 log config 
# less detail

handler = logging.FileHandler(get_time())  # Or FileHandler or anything else
# Configure the fields to include in the JSON output. message is the main log string itself
format_str = '%(message)%(levelname)%(name)%(asctime)'
formatter = jsonlogger.JsonFormatter(format_str)
handler.setFormatter(formatter)
logger = logging.getLogger(__name__)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
# Normally we would attach the handler to the root logger, and this would be unnecessary
logger.propagate = False
